In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import torch
import torchvision
from torchvision import datasets
from torchvision import transforms as T # for simplifying the transforms
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split
from torchvision import models

print(torch.__version__)

import timm
from timm.loss import LabelSmoothingCrossEntropy 

import matplotlib.pyplot as plt
%matplotlib inline
import sys
from tqdm import tqdm
import time
import copy
import warnings
warnings.filterwarnings("ignore")

import pathlib

2.0.0+cpu


c:\Users\DELL\miniconda3\envs\plant_disease\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
PRJ_ROOT = pathlib.Path(os.getcwd()).parent
DATA_PATH = os.path.join(
    PRJ_ROOT, 'data', 'rice_leaf_diseases'
)

HUB_URL = "SharanSMenon/swin-transformer-hub:main"
MODEL_NAME = "swin_tiny_patch4_window7_224"

TRAIN_BATCH = 1
TEST_BATCH = 1

NUM_WORKERS = 1
NUM_EPOCHS = 10

In [3]:
def get_classes(data_dir):
    all_data = datasets.ImageFolder(data_dir)
    return all_data.classes


def get_data_loaders(data_dir, batch_size, num_workers, train = False):
    if train:
        #train
        transform = T.Compose([
            T.RandomHorizontalFlip(),
            T.RandomVerticalFlip(),
            T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.25),
            T.Resize(256),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize(timm.data.IMAGENET_DEFAULT_MEAN, timm.data.IMAGENET_DEFAULT_STD), # imagenet means
            T.RandomErasing(p=0.1, value='random')
        ])
        train_data = datasets.ImageFolder(os.path.join(data_dir, "train/"), transform = transform)
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
        return train_loader, len(train_data)


    else:
        # val/test
        transform = T.Compose([ # We dont need augmentation for test transforms
            T.Resize(256),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize(timm.data.IMAGENET_DEFAULT_MEAN, timm.data.IMAGENET_DEFAULT_STD), # imagenet means
        ])

        val_data = datasets.ImageFolder(os.path.join(data_dir, "test/"), transform=transform)
        test_data = datasets.ImageFolder(os.path.join(data_dir, "test/"), transform=transform)
        val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
        test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
        return val_loader, test_loader, len(val_data), len(test_data)

In [4]:
(train_loader, train_data_len) = get_data_loaders(DATA_PATH, 
                                                  TRAIN_BATCH,
                                                  NUM_WORKERS,
                                                  train=True)
(val_loader, test_loader, valid_data_len, test_data_len) = get_data_loaders(DATA_PATH,
                                                                            TEST_BATCH,
                                                                            NUM_WORKERS,
                                                                            train=False)

In [5]:
classes = get_classes(os.path.join(DATA_PATH, "train/"))
print(classes, len(classes))

['Bacterial leaf blight', 'Brown spot', 'Leaf smut'] 3


In [26]:
len(classes)

3

In [6]:
dataloaders = {
    "train": train_loader,
    "val": val_loader
}
dataset_sizes = {
    "train": train_data_len,
    "val": valid_data_len
}

In [7]:
print(len(train_loader), len(val_loader), len(test_loader))

96 24 24


In [8]:
# now, for the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [1]:
torch.device('cpu')

NameError: name 'torch' is not defined

torch.device('cpu)

In [9]:
def init_model(hub_url:str, model_name:str):
    model = torch.hub.load(hub_url, model_name, pretrained=True)
    for param in model.parameters(): #freeze model
        param.requires_grad = False

    n_inputs = model.head.in_features
    model.head = nn.Sequential(
        nn.Linear(n_inputs, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, len(classes))
    )
    return model

In [10]:
import pytorch_lightning as pl
from torchmetrics.functional import accuracy
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger
try:
    from torchmetrics.functional import accuracy
except ImportError:
    from pytorch_lightning.metrics.functional import accuracy

import mlflow.pytorch
from mlflow import MlflowClient

In [11]:
from lightning.pytorch import Trainer
from lightning.pytorch.loggers import MLFlowLogger

mlf_logger = MLFlowLogger(experiment_name="lightning_logs", tracking_uri="file:./mlruns")
trainer = Trainer(logger=mlf_logger)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [12]:
from torchmetrics.classification import MulticlassAccuracy

In [13]:
from pytorch_lightning.utilities.types import TRAIN_DATALOADERS


class PlantDiseasesModel(pl.LightningModule):
    def __init__(self, hub_url:str, model_name:str, lr:float=0.01):
        super().__init__()
        self.save_hyperparameters()
        self.model = init_model(hub_url, model_name)
        self.criterion = LabelSmoothingCrossEntropy()

    def forward(self, x):
        # self.model.train()
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        self.log("train_loss", loss)
        return loss

    def evaluate(self, batch, stage=None):
        x, y = batch
        logits = self.model(x)
        loss = self.criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = MulticlassAccuracy(num_classes=len(classes), average="macro")(preds, y)
        if stage:
            self.log(f"{stage}_loss", loss, prog_bar=True)
            self.log(f"{stage}_acc", acc, prog_bar=True)

    def validation_step(self, batch, batch_idx, dataloader_idx=0):
        self.evaluate(batch, "val")

    def test_step(self, batch, batch_idx):
        self.evaluate(batch, "test")

    

    def configure_optimizers(self):
        optimizer = optim.AdamW(
            self.model.head.parameters(),
            lr=self.hparams.lr,
            weight_decay=5e-4,
        )
        steps_per_epoch = 45000 // TRAIN_BATCH
        scheduler_dict = {
            "scheduler": torch.optim.lr_scheduler.OneCycleLR(
                optimizer,
                0.1,
                epochs=self.trainer.max_epochs,
                steps_per_epoch=steps_per_epoch,
            ),
            "interval": "step",
        }
        return {"optimizer": optimizer, "lr_scheduler": scheduler_dict} 
    

In [14]:
model = PlantDiseasesModel(HUB_URL, MODEL_NAME)

Using cache found in C:\Users\DELL/.cache\torch\hub\SharanSMenon_swin-transformer-hub_main


In [22]:
os.makedirs("../models/checkpoints", exist_ok=True)
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath="../models/checkpoints",
    filename="best-checkpoint {epoch:02d}",
    save_top_k=1,
    verbose=True,
    auto_insert_metric_name=False)


trainer = pl.Trainer(
    max_epochs=NUM_EPOCHS,
    # accelerator="auto",
    logger=CSVLogger(save_dir="../logs/"),
    callbacks=[checkpoint_callback, LearningRateMonitor(logging_interval="step"), TQDMProgressBar(refresh_rate=10)],
    default_root_dir="../models/checkpoints/checkpoints.pth"
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [23]:
def print_auto_logged_info(r):
    tags = {k: v for k, v in r.data.tags.items() if not k.startswith("mlflow.")}
    artifacts = [f.path for f in MlflowClient().list_artifacts(r.info.run_id, "model")]
    print("run_id: {}".format(r.info.run_id))
    print("artifacts: {}".format(artifacts))
    print("params: {}".format(r.data.params))
    print("metrics: {}".format(r.data.metrics))
    print("tags: {}".format(tags))

In [24]:

mlflow.pytorch.autolog()

# Train the model
with mlflow.start_run() as run:
    trainer.fit(model,
                train_dataloaders=dataloaders["train"],
                val_dataloaders=dataloaders["val"])

# fetch the auto logged parameters and metrics
print_auto_logged_info(mlflow.get_run(run_id=run.info.run_id))

2023/06/11 19:08:55 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.

  | Name      | Type                       | Params
---------------------------------------------------------
0 | model     | SwinTransformer            | 27.9 M
1 | criterion | LabelSmoothingCrossEntropy | 0     
---------------------------------------------------------
395 K     Trainable params
27.5 M    Non-trainable params
27.9 M    Total params
111.658   Total estimated model params size (MB)


Epoch 9: 100%|██████████| 96/96 [00:35<00:00,  2.68it/s, v_num=0, val_loss=0.898, val_acc=0.250]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 96/96 [00:36<00:00,  2.62it/s, v_num=0, val_loss=0.898, val_acc=0.250]


2023/06/11 19:16:11 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\DELL\AppData\Local\Temp\tmpsivgbyy9\model\data, flavor: pytorch), fall back to return ['torch==2.0.0', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback.


run_id: dbe262513c6a424b8068003c86545b74
artifacts: ['model/MLmodel', 'model/conda.yaml', 'model/data', 'model/python_env.yaml', 'model/requirements.txt']
params: {'amsgrad': 'False', 'betas': '(0.9, 0.999)', 'capturable': 'False', 'differentiable': 'False', 'epochs': '10', 'eps': '1e-08', 'foreach': 'None', 'fused': 'None', 'lr': '0.01', 'maximize': 'False', 'optimizer_name': 'AdamW', 'weight_decay': '0.0005'}
metrics: {'train_loss': 0.4141537547111511, 'val_acc': 0.2500000298023224, 'val_loss': 0.8976366519927979}
tags: {'Mode': 'training'}


In [25]:
trainer.test(model, dataloaders=dataloaders['val'])

Testing DataLoader 0: 100%|██████████| 24/24 [00:10<00:00,  2.38it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.2500000298023224     │
│         test_loss         │    0.8976365923881531     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.8976365923881531, 'test_acc': 0.2500000298023224}]

In [19]:
print_auto_logged_info(mlflow.get_run(run_id=run.info.run_id))

run_id: 0a82b94c01bb4db091c071f88aecd3d0
artifacts: ['model/MLmodel', 'model/conda.yaml', 'model/data', 'model/python_env.yaml', 'model/requirements.txt']
params: {'amsgrad': 'False', 'betas': '(0.9, 0.999)', 'capturable': 'False', 'differentiable': 'False', 'epochs': '3', 'eps': '1e-08', 'foreach': 'None', 'fused': 'None', 'lr': '0.01', 'maximize': 'False', 'optimizer_name': 'AdamW', 'weight_decay': '0.0005'}
metrics: {'test_acc': 0.2777778208255768, 'test_loss': 0.5474447011947632, 'train_loss': 0.3175453245639801, 'val_acc': 0.2777778208255768, 'val_loss': 0.5474446415901184}
tags: {'Mode': 'testing'}
